In [1]:
import torch
from torch.nn import *
import torch.nn.functional as F
from pandas_datareader import data as web
import os
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from diebold_mariano import diebold_mariano_test

In [2]:
from utils import *

DEVICE:  cuda


In [3]:
T=30

In [4]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib

# Set consistent style for publication-quality plots
sns.set(style="whitegrid", context="talk", palette="colorblind")
matplotlib.rcParams.update({
    "axes.titlesize": 18,
    "axes.labelsize": 12,
    "xtick.labelsize": 10,
    "ytick.labelsize": 10,
    "legend.fontsize": 10,
    "legend.frameon": True,
    "axes.titleweight": "bold"
})

def customize_ax(ax, title=None, xlabel=None, ylabel=None):
    if title:
        ax.set_title(title, fontsize=20, fontweight="bold")
    if xlabel:
        ax.set_xlabel(xlabel, fontsize=12)
    if ylabel:
        ax.set_ylabel(ylabel, fontsize=12)
    ax.tick_params(axis='both', which='major', labelsize=10)
    ax.grid(False)
    if ax.get_legend():
        ax.legend(fontsize=10, frameon=True)

# Insert your original loop code here afterward, using customize_ax() and matplotlib.rcParams updates
# on all relevant plots to ensure publication-quality formatting.


In [5]:
etfs = ['SPY','XLB','XLE','XLF','XLI','XLK','XLP','XLV','XLY','XLU']
metric2 = "SD_Squared_Returns"
metric3 = "MAD_Returns"
metric4 = "IQR"
metric5 = "SD_Returns"
metrics = ["ATR",metric3,metric4,metric5]
# start = '2001-01-01'
# end = '2020-01-01'
# start_t = '2022-01-02'
# end_t = '2024-01-05'
date_list = [['2023-01-05', '2024-01-05', "2024-01-05", "2025-01-05"], ['2019-01-05', '2024-01-05', "2024-01-05", "2025-01-05"], ['2014-01-05', "2024-01-05", "2024-01-05", "2025-01-05"], ['2004-01-05', '2024-01-05', "2024-01-05", "2025-01-05"], ['1994-01-05', '2024-01-05', "2024-01-05", "2025-01-05"]]
starts,ends = np.array([(f'200{i}-01-01',f'200{i+12}-01-01') for i in range(0,12,4)]).T
models = {}
lrs = [0]
h1 = 50
h2 = 50
hl1s = [20,50,70,100]
hl1s = [20,50,70,100]
models = ["LSTM","RNN"]
b_types = ['ATR' ,'STD']
lr = 0
feat_combos = [['Open','Close','High','Low','Volume','Return','Short_MA','Long_MA'],['Open','Close','High','Low','Volume'],['index','Open','Close','High','Low','Volume']]

In [ ]:
num=20000
for file in os.listdir('../output'):
    if "runtest" in file:
        num+=1
os.mkdir(f'../output/runtest_{num}')
for d,(start,end,start_t,end_t) in enumerate(date_list):
    os.mkdir(f'../output/runtest_{num}/date_config{d}')
    for model_type in models:
        os.mkdir(f'../output/runtest_{num}/date_config{d}/{model_type}')
        root = f'../output/runtest_{num}/date_config{d}/{model_type}'
        with open(os.path.join(root,'run_data.txt'),'w') as f:
            f.write(f'Start Training Date: {start} \n End Training Date: {end} \n Start Sim Date: {start_t} \n End Sim Date: {end_t} \n')
            f.write(f'\n Model Type: {model_type}')
            f.write(f'Look-back window: {T} days')

        for etf in etfs:
            os.mkdir(f'{root}/{etf}')
        for lr in lrs:
            results = pd.DataFrame({"ETF":[],"Metric":[],"Final_Value":[],"Passive_Potential":[],"Relative_MSE":[],"MAPE":[],"DM_stat":[],"DM_p":[]})
            money_df_std = pd.DataFrame()
            money_df_atr = pd.DataFrame()
            money_df_sd_prices = pd.DataFrame()
            money_df_iqr = pd.DataFrame()
            money_df_passive = pd.DataFrame()
            for j,etf in enumerate(etfs):
                #fig,ax = plt.subplots(1,len(metrics),figsize=(26,6))
                fig,ax = plt.subplots(2,2,figsize=(12,9))
                atr_preds = []
                for i,metric in enumerate(metrics):
                    scaler = StandardScaler()
                    scaler_x = StandardScaler()
                    df = get_cleaned_df(etf,start,end)
                    #print(df)
                    model,fig_tr,fig_te = get_trained_model(df,scaler, scaler_x, metric=metric,model_type=model_type)
                    fig_tr.savefig(os.path.join(root,etf,f'training_results_{metric}'))
                    fig_te.savefig(os.path.join(root,etf,f'testing_results_{metric}'))
                    df_test = get_cleaned_df(etf,start_t,end_t)
                    ini_cash = 10000
                    final_value, money, shares,passive_value,buy_points,sell_points,preds, t_money,p_money,lowers,uppers,vols = backtest_strategy_mr(df_test,model,scaler,scaler_x,metric,sell_scale=2,buy_scale=2)
                    fig_vol,ax_vol = plt.subplots(figsize=(10,5))
                    sns.lineplot(x = range(0,len(df_test.index)-(T+1)),y=lowers,color = 'blue',ax=ax_vol,label = "Buy")
                    sns.lineplot(x = range(0,len(df_test.index)-(T+1)),y=uppers,color = 'red',ax=ax_vol,label="Sell")
                    sns.lineplot(x = range(0,len(df_test.index)-(T+1)),y=vols,color = 'black',ax=ax_vol,label=f"{metric}")
                    ax_vol.set_title(f"Predicted {metric} and Buy/Sell Signals")
                    ax_vol.set_ylabel(f"{metric}")
                    ax_vol.set_xlabel(f"Days after {start_t}")
                    for buy in buy_points:
                        ax_vol.axvline(buy,color = 'Orange',alpha=.5,)
                    for sell in sell_points:
                        ax_vol.axvline(sell,color='Purple',alpha=.5,)
                    #ax_vol.legend()
                    customize_ax(ax_vol)
                    
                    fig_vol.tight_layout()
                    fig_vol.savefig(os.path.join(root,etf,f'{metric}_signals.png'))
                    

                    print(f"{etf} Final Value: {final_value}, Final Money: {money}, Final Shares: {shares}")
                    
                    real_met = np.array(df_test[metric].iloc[T:len(df_test)-1])
                    atr_met = np.array(df_test["ATR"].iloc[T:len(df_test)-1])
                    null_model_error = np.sum((real_met-real_met.mean())**2)
                    mse_error = 1/len(real_met) * np.sum((real_met-preds)**2)/null_model_error
                    mape_error = 1/len(real_met) * np.sum(abs(real_met-preds)/(real_met+1e-8))
                    if metric=="ATR": #atr should go first
                        money_df_atr[etf] = t_money
                        atr_preds = preds
                    elif metric==metric3:
                        money_df_std[etf] = t_money
                        money_df_passive[f"{etf}_Passive"] = p_money
                    elif metric == metric5:
                        money_df_sd_prices[etf] = t_money
                    elif metric == metric4:
                        money_df_iqr[etf] = t_money

                    dm,p = diebold_mariano_test(atr_met,atr_preds,real_met,preds,loss="mae")
                    results.loc[len(results)] = [etf,metric,final_value,passive_value,mse_error,mape_error,dm,p]

                    sns.lineplot(preds,ax=ax[i//2][i%2],label=f"Prediction {metric}",color="Blue",linestyle = '--')
                    sns.lineplot(real_met,ax=ax[i//2][i%2],label = f"True {metric}",color="Orange")
                    sns.despine(ax=ax[i//2][i%2])
                    #for buy in buy_points:
                        #ax[i].axvline(buy,color="Green",alpha=.6)
                    #for sell in sell_points:
                        #ax[i].axvline(sell,color="Red",alpha=.6)
                    ax[i//2][i%2].set_title(f"{etf} - {metric} ",fontsize = 20)
                    ax[i//2][i%2].set_xlabel(f'Total MSE Error {mse_error:.6f} | Total MAPE Error: {mape_error:.3f}')
                    customize_ax(ax[i//2][i%2])
                    #plt.title(f"Performance of LSTM on {etf}")
                fig.supylabel("Volatility Metric")
                fig.supxlabel(f"Days after {start_t}")
                fig.tight_layout()
                fig.savefig(os.path.join(root,etf,'predictions.png'))
                fig_sim,ax_sim = plt.subplots(figsize=(10,6))
                sns.lineplot(x=range(0, len(df_test.index)-(T+1)), y=money_df_atr[etf], label="ATR",ax=ax_sim,palette = 'colorblind')
                sns.lineplot(x=range(0,len(df_test.index)-(T+1)),y=money_df_std[etf],label="SD_Returns",ax=ax_sim,palette = 'colorblind')
                sns.lineplot(x=range(0,len(df_test.index)-(T+1)),y=money_df_sd_prices[etf],label="MAD_Returns",ax=ax_sim,palette = 'colorblind')
                sns.lineplot(x=range(0,len(df_test.index)-(T+1)),y=money_df_passive[f"{etf}_Passive"],label="Passive",ax=ax_sim,palette = 'colorblind')
                sns.lineplot(x=range(0,len(df_test.index)-(T+1)),y=money_df_iqr[etf],label="IQR",ax=ax_sim,palette = 'colorblind')
                sns.despine(ax=ax_sim)
                ax_sim.axhline(1000, label = "Initial",color="Black")
                ax_sim.set_xlabel(f"Days after {start_t}")
                ax_sim.set_ylabel(f"Value ($)")
                ax_sim.set_title(f"ETF Performance Over Time for {etf}")
                customize_ax(ax_sim)
                ax_sim.legend(
                    loc='upper left',              # anchor the legend’s corner
                    bbox_to_anchor=(1.02, 1.0),     # (x, y) in axes fraction coordinates
                    borderaxespad=0
                )
                fig_sim.tight_layout()
                fig_sim.savefig(os.path.join(root,etf,'simulation_perf_lr_{lr}'))
                
            fig2,ax2 = plt.subplots(figsize = (8,6))
            sns.barplot(data=results,x="ETF",y="Final_Value",hue="Metric",ax=ax2,palette='colorblind') 
            sns.despine(ax=ax2)   
            plt.legend(loc=(1.1,1))
            fig2.tight_layout()
            customize_ax(ax2)
            fig2.savefig(os.path.join(root,'return_metrics_lr_{lr}.png'))
            results.to_csv(os.path.join(root,'overall_results.csv'),index=False)

[*********************100%***********************]  1 of 1 completed


read  251  lines of data for ticker:  SPY


[*********************100%***********************]  1 of 1 completed


read  251  lines of data for ticker:  SPY
On the 83th day, Bought 0.9437879874664956 shares for $500.0
On the 128th day, sold 0.9437879874664956 shares for $523.4720072482918
On the 179th day, Bought 0.8789508842245896 shares for $500.0
On the 180th day, Bought 0.8805607410799197 shares for $500.0
On the 206th day, sold 1.7595116253045093 shares for $1052.5222591409045
On the 223th day, Bought 0.8228285554421881 shares for $500.0
On the 224th day, Bought 0.8241848811525402 shares for $500.0
On the 238th day, sold 1.6470134365947282 shares for $990.4150599618739


[*********************100%***********************]  1 of 1 completed

...

[*********************100%***********************]  1 of 1 completed

On the 226th day, sold 9.387350501023597 shares for $853.0285400280144
On the 241th day, Bought 5.837030119075415 shares for $500.0
XLE Final Value: 964.7631002149055, Final Money: 454.19807569937893, Final Shares: 5.837030119075415


read  1258  lines of data for ticker:  XLE


[*********************100%***********************]  1 of 1 completed


read  251  lines of data for ticker:  XLE


In [3]:
os.listdir()

NameError: name 'os' is not defined

In [ ]:
for i in range(32,41):
    df = pd.read_csv(f'runtest_{i}/overall_results.csv')
    print(i,df.groupby('Metric')['Relative_MSE'].mean())

In [ ]:
scaler_x = StandardScaler()
scaler = StandardScaler()
start,end,start_t,end_t = date_list[1]
df_xlp = get_cleaned_df('XLP',start,end)
df_xlp_test = get_cleaned_df('XLP',start_t,end_t)
model,fig_tr,fig_te = get_trained_model(df_xlp,scaler, scaler_x, metric="SD_Squared_Returns",model_type="LSTM")
final_value, money, shares,passive_value,buy_points,sell_points,preds, t_money,p_money = backtest_strategy_mr(df_xlp_test,model,scaler,scaler_x,"SD_Squared_Returns",sell_scale=1.7,buy_scale=1.7)

In [ ]:
sns.lineplot(t_money)